In [15]:
import pandas as pd
from sqlalchemy import create_engine
from random import randint
import pymongo
from datetime import datetime
import math

In [2]:
main_file = "../static/data/US_Accidents_Dec19.csv"

In [3]:
main_df = pd.read_csv(main_file)

In [9]:
main_df['Start_Time'] = main_df['Start_Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [10]:
main_df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


### Transform Accident DataFrame

In [5]:
# List conversion function.
def convert(s):
    lst = []
    temp = str(s).split(", ")
    
    for x in temp:
        lst.append(x.translate({ord(i): None for i in "[']"}))
    
    return lst

In [6]:
main_df.columns

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [8]:
main_cols = ["ID", "Severity", "Start_Lat", "Start_Lng", "Description", "City", "County", "State", "Zipcode"]
main_trans_df = main_df[main_cols].copy()
main_trans_df['Year'] = ''
main_trans_df['Month'] = ''
main_trans_df['Day'] = ''
main_trans_df['Hour'] = ''
main_trans_df['Minutes'] = ''
main_trans_df['Seconds'] = ''
main_trans_df = main_trans_df.rename(columns={"Start_Lat": "Lat", "Start_Lng": "Lng"})
main_trans_df.head()

,ID,Severity,Lat,Lng,Description,City,County,State,Zipcode,Year,Month,Day,Hour,Minutes,Seconds
0,A-1,3,39.865147,-84.058723,Right lane blocked due to accident on I-70 Eas...,Dayton,Montgomery,OH,45424,,,,,,
1,A-2,2,39.928059,-82.831184,Accident on Brice Rd at Tussing Rd. Expect del...,Reynoldsburg,Franklin,OH,43068-3402,,,,,,
2,A-3,2,39.063148,-84.032608,Accident on OH-32 State Route 32 Westbound at ...,Williamsburg,Clermont,OH,45176,,,,,,
3,A-4,3,39.747753,-84.205582,Accident on I-75 Southbound at Exits 52 52B US...,Dayton,Montgomery,OH,45417,,,,,,
4,A-5,2,39.627781,-84.188354,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Dayton,Montgomery,OH,45459,,,,,,


In [ ]:
main_trans_df['Year'] = main_df['Start_Time'].apply(lambda x: x.year)
main_trans_df['Month'] = main_df['Start_Time'].apply(lambda x: x.month)
main_trans_df['Day'] = main_df['Start_Time'].apply(lambda x: x.day)
main_trans_df['Hour'] = main_df['Start_Time'].apply(lambda x: x.hour)
main_trans_df['Minutes'] = main_df['Start_Time'].apply(lambda x: x.minute)
main_trans_df['Seconds'] = main_df['Start_Time'].apply(lambda x: x.second)
main_trans_df.head()

### Create Data Frames

In [14]:
short2016_df = main_trans_df.loc[main_trans_df['Year'] == 2016,:]
short2017_df = main_trans_df.loc[main_trans_df['Year'] == 2017,:]
short2018_df = main_trans_df.loc[main_trans_df['Year'] == 2018,:]
short2019_df = main_trans_df.loc[main_trans_df['Year'] == 2019,:]
print(len(short2016_df))
print(len(short2017_df))
print(len(short2018_df))
print(len(short2019_df))

410600
717483
892615
953630


In [69]:
# count2016 = 0
# count2017 = 0
# count2018 = 0
# count2019 = 0
# for i in range(len(main_trans_df)):
#     date = datetime.strptime(main_trans_df['Start_Time'][i], '%Y-%m-%d %H:%M:%S')
#     if (date.year == 2016):
#         count2016 = count2016 + 1
#     elif (date.year == 2017):
#         count2017 = count2017 + 1
#     elif (date.year == 2018):
#         count2018 = count2018 + 1
#     elif (date.year == 2019):
#         count2019 = count2019 + 1
#     else :
#         pass
# print(count2016)
# print(count2017)
# print(count2018)
# print(count2019)

410600
717483
892615
953630


In [19]:
index_2016 = [randint(0,len(short2016_df)-1) for _ in range(math.floor(len(short2016_df)/9.5))]
index_2017 = [randint(0,len(short2017_df)-1) for _ in range(math.floor(len(short2017_df)/9.5))]
index_2018 = [randint(0,len(short2018_df)-1) for _ in range(math.floor(len(short2018_df)/9.5))]
index_2019 = [randint(0,len(short2019_df)-1) for _ in range(math.floor(len(short2019_df)/9.5))]
index_2016 = list(dict.fromkeys(index_2016))
index_2017 = list(dict.fromkeys(index_2017))
index_2018 = list(dict.fromkeys(index_2018))
index_2019 = list(dict.fromkeys(index_2019))
print(len(index_2016))
print(len(index_2017))
print(len(index_2018))
print(len(index_2019))

40971
71513
89261
95269


In [20]:
sshort2016_df = short2016_df.iloc[index_2016,:]
sshort2017_df = short2017_df.iloc[index_2017,:]
sshort2018_df = short2018_df.iloc[index_2018,:]
sshort2019_df = short2019_df.iloc[index_2019,:]
# print(len(sshort2016_df))
# print(len(sshort2017_df))
# print(len(sshort2018_df))
# print(len(sshort2019_df))

In [127]:
sshort_list = [sshort2016_df,sshort2017_df,sshort2018_df,sshort2019_df]
sshort_df = pd.concat(sshort_list, axis=0, join='outer', ignore_index=True, copy=True)
print(len(sshort_df))
sshort_df.head()

39698


,ID,Severity,Start_Time,End_Time,Lat,Lng,Description,Number,Street,Side,City,County,State,Zipcode
0,A-191062,2,2016-08-24 12:50:40,2016-08-24 13:50:40,42.984879,-85.677193,Accident on US-131 Bus Leonard St at US-131.,NaN,Leonard St NW,R,Grand Rapids,Kent,MI,49504
1,A-158996,2,2016-07-11 14:42:37,2016-07-11 15:57:37,33.615955,-81.715523,Accident on SC-26 Reynolds Pond Rd at Pipeline...,254.0,Pipeline Rd,L,Aiken,Aiken,SC,29801-7710
2,A-15041,2,2016-11-04 12:55:00,2016-11-04 13:24:37,37.765965,-122.404976,#2 lane blocked due to accident on 16th St Eas...,1756.0,16th St,R,San Francisco,San Francisco,CA,94103-5010
3,A-66856,2,2016-10-25 06:27:35,2016-10-25 07:00:00,33.574562,-117.633430,Right hand shoulder blocked due to accident on...,NaN,Benjamin Dr,R,Ladera Ranch,Orange,CA,92694
4,A-127969,2,2016-11-16 09:13:49,2016-11-16 09:43:38,27.389076,-82.559334,Accident on US-41 Tamiami Trl at Edwards Dr.,NaN,Edwards Dr,R,Sarasota,Sarasota,FL,34243


In [37]:
# main_trans_df.to_csv("../static/data/US_Accidents_Dec19_Transformed.csv")
short_df.to_csv("../static/data/Test_Data2.csv")

In [103]:
short2016_df.to_csv("../static/data/main2016.csv")
short2017_df.to_csv("../static/data/main2017.csv")
short2018_df.to_csv("../static/data/main2018.csv")
short2019_df.to_csv("../static/data/main2019.csv")

In [21]:
sshort2016_df.to_csv("../static/data/short2016.csv")
sshort2017_df.to_csv("../static/data/short2017.csv")
sshort2018_df.to_csv("../static/data/short2018.csv")
sshort2019_df.to_csv("../static/data/short2019.csv")

### Create database connection

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db1 = client.Accidents
db2 = client.Accidents_Short

### Drop Documents in Databases if Necessary

In [43]:
db1.accident.drop()
db2.accident.drop()

### Load DataFrames into database

In [44]:
db1.accident.insert_many(main_trans_df.to_dict('records'))
db2.accident.insert_many(short_df.to_dict('records'))